In [1]:
import cma_gui as cma
import pandas as pd

# Run All Calcs

In [2]:
# Check if new data needs to be pulled
date_check = pd.read_excel('P:\\Advisory\\Research\\Automation\\CMAs\\Data\\bloomberg_data_us.xlsx', index_col=0)

if (cma.val_dict['as_of_date'] in date_check.index):
    import backfill_calc
    import std_dev
    import fixed_income_calcs
    import equity_calcs
    import alts
    print('No new data needed')
else:
    import data_pull
    import backfill_calc
    import std_dev
    import fixed_income_calcs
    import equity_calcs
    import alts
    print('New data pulled')

No new data needed


# Final Data

## US

In [3]:
df_final_us = pd.concat([equity_calcs.equity_returns_us, fixed_income_calcs.fixed_returns_us*100, alts.alts_returns_us])
df_final_us = pd.DataFrame(df_final_us)
df_final_us = df_final_us.rename(columns={0: "Expected Return"})

# Combine Expected Returns with Standard Deviation
df_final_us = df_final_us.merge(std_dev.annual_adj_std_dev_us*100, left_index=True, right_index=True)
df_final_us = df_final_us.rename(columns={0: "Standard Deviation"})

# Convert to ints and round
df_final_us['Expected Return'] = pd.to_numeric(df_final_us['Expected Return']).round(1)
df_final_us['Standard Deviation'] = pd.to_numeric(df_final_us['Standard Deviation']).round(1)

# Reorder
equity_us_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if 'equity_us_name' in k}.values())))
fixed_us_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if 'fixed_us_name' in k}.values())))
alts_us_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if 'alts_us_name' in k}.values())))

order_name_us = equity_us_name + fixed_us_name + alts_us_name
df_final_us = df_final_us.reindex(index=order_name_us)

df_final_us

,Expected Return,Standard Deviation
U.S. Equity,6.3,15.3
U.S. Large Cap Equity,6.2,15.1
U.S. Mid Cap Equity,6.6,17.0
U.S. Small Cap Equity,7.0,20.0
U.S. Micro Cap Equity,6.8,20.8
Global Equity,6.5,15.3
International Developed Equity,6.2,16.5
International Small Cap Equity,6.2,17.4
Global Emerging Markets Equity,8.5,20.8
U.S. Aggregate,2.7,3.4


## NonUS

In [4]:
df_final_nonus = pd.concat([equity_calcs.equity_returns_nonus, fixed_income_calcs.fixed_returns_nonus*100, alts.alts_returns_nonus])
df_final_nonus = pd.DataFrame(df_final_nonus)

df_final_nonus = df_final_nonus.rename(columns={0: "Expected Return"})
    
# Combine Expected Returns with Standard Deviation
df_final_nonus = df_final_nonus.merge(std_dev.annual_adj_std_dev_nonus*100, left_index=True, right_index=True)
df_final_nonus = df_final_nonus.rename(columns={0: "Standard Deviation"})

# Add Income Return

df_final_nonus = df_final_nonus.join(pd.DataFrame(fixed_income_calcs.income_return_nonus*100)).rename(columns={0: "Income Return"})

df_final_nonus.loc[['U.S. Equity', 'U.S. Small Cap Equity'], 'Income Return'] = cma.val_dict['us_equity_income']
df_final_nonus.loc[['Europe Ex-UK Equity', 'Europe Small Cap Equity'], 'Income Return'] = cma.val_dict['europe_ex_uk_equity_income']
df_final_nonus.loc['UK Equity', 'Income Return'] = cma.val_dict['uk_equity_income']
df_final_nonus.loc[['Japan Equity', 'Japan Small Cap Equity'], 'Income Return'] = cma.val_dict['japan_equity_income']
df_final_nonus.loc['Developed Market Pacific Ex-Japan Equity', 'Income Return'] = cma.val_dict['apac_ex_japan_equity_income']
df_final_nonus.loc['Global Emerging Markets Equity', 'Income Return'] = cma.val_dict['em_equity_income']

df_final_nonus.loc[list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if 'alts_nonus_name' in k}.values()))),'Income Return'] = 0


# Convert to ints and round
df_final_nonus['Expected Return'] = pd.to_numeric(df_final_nonus['Expected Return']).round(1)
df_final_nonus['Standard Deviation'] = pd.to_numeric(df_final_nonus['Standard Deviation']).round(1)
df_final_nonus['Income Return'] = pd.to_numeric(df_final_nonus['Income Return']).round(1)

# Reorder
equity_nonus_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if 'equity_nonus_name' in k}.values())))
fixed_nonus_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if 'fixed_nonus_name' in k}.values())))
alts_nonus_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if 'alts_nonus_name' in k}.values())))

order_name_nonus = equity_nonus_name + fixed_nonus_name + alts_nonus_name
df_final_nonus = df_final_nonus.reindex(index=order_name_nonus)

df_final_nonus

,Expected Return,Standard Deviation,Income Return
U.S. Equity,6.2,14.8,1.9
U.S. Small Cap Equity,6.7,19.3,1.9
Europe Ex-UK Equity,6.5,16.6,3.1
Europe Small Cap Equity,6.2,17.7,3.1
UK Equity,7.0,13.1,3.5
Japan Equity,4.4,14.5,1.7
Japan Small Cap Equity,4.3,16.2,1.7
Developed Market Pacific Ex-Japan Equity,8.2,17.8,3.8
Global Emerging Markets Equity,8.4,19.1,2.5
Global Developed Market Sovereign Fixed Income,0.7,9.7,2.6


In [5]:
std_dev.corr_matrix_final_nonus

,U.S. Equity,U.S. Small Cap Equity,Europe Ex-UK Equity,Europe Small Cap Equity,UK Equity,Japan Equity,Japan Small Cap Equity,Developed Market Pacific Ex-Japan Equity,Global Emerging Markets Equity,Global Developed Market Sovereign Fixed Income,...,Global High Yield Fixed Income,Emerging Markets Local Currency Fixed Income,Emerging Markets Sovereign Fixed Income USD,Emerging Markets Corporate Fixed Income,Emerging Debt Agg USD,Long-Short Equity,Global Macro,USD_Long-Short Equity,USD_Global Macro,USD_U.S. Equity
U.S. Equity,1.000000,0.867025,0.758463,0.684587,0.749483,0.644667,0.506887,0.704459,0.674791,0.163901,...,0.647048,0.493602,0.476644,0.489716,0.493392,0.798928,0.339677,0.584021,0.196544,0.793727
U.S. Small Cap Equity,0.867025,1.000000,0.694252,0.709171,0.660399,0.557648,0.445013,0.665164,0.640045,0.041618,...,0.589041,0.404375,0.377179,0.408013,0.399916,0.780098,0.241555,0.690840,0.209246,0.737313
Europe Ex-UK Equity,0.758463,0.694252,1.000000,0.894972,0.865785,0.576112,0.406162,0.755727,0.756324,0.036211,...,0.564364,0.471279,0.370681,0.394578,0.378680,0.586901,0.107524,0.705629,0.311032,0.761973
Europe Small Cap Equity,0.684587,0.709171,0.894972,1.000000,0.832281,0.538900,0.415266,0.742853,0.746877,-0.080345,...,0.525373,0.340739,0.272346,0.338949,0.292266,0.562801,0.002723,0.794864,0.294744,0.754817
UK Equity,0.749483,0.660399,0.865785,0.832281,1.000000,0.550552,0.380941,0.761756,0.720069,-0.038276,...,0.539376,0.408019,0.341523,0.367047,0.345802,0.572271,0.080314,0.703804,0.277959,0.760886
Japan Equity,0.644667,0.557648,0.576112,0.538900,0.550552,1.000000,0.875482,0.577217,0.579271,0.187303,...,0.530183,0.450953,0.424233,0.428490,0.436964,0.647872,0.307032,0.458835,0.197579,0.475299
Japan Small Cap Equity,0.506887,0.445013,0.406162,0.415266,0.380941,0.875482,1.000000,0.461528,0.450772,0.315222,...,0.451659,0.441817,0.413065,0.385452,0.418248,0.574629,0.352799,0.299140,0.158491,0.298378
Developed Market Pacific Ex-Japan Equity,0.704459,0.665164,0.755727,0.742853,0.761756,0.577217,0.461528,1.000000,0.881289,0.136967,...,0.617278,0.617321,0.473286,0.495784,0.481869,0.639421,0.159496,0.686595,0.291716,0.665606
Global Emerging Markets Equity,0.674791,0.640045,0.756324,0.746877,0.720069,0.579271,0.450772,0.881289,1.000000,0.120465,...,0.624693,0.638340,0.475907,0.494802,0.481215,0.647191,0.168517,0.719431,0.346225,0.650731
Global Developed Market Sovereign Fixed Income,0.163901,0.041618,0.036211,-0.080345,-0.038276,0.187303,0.315222,0.136967,0.120465,1.000000,...,0.511381,0.677410,0.718982,0.574115,0.701642,0.420651,0.764710,-0.386190,0.136654,-0.321894
